# Bollinger Bands

## Load data

In [ ]:
import pandas as pd

df_base = pd.read_parquet('../../../data/SPY.parquet')
df_base

,Open,High,Low,Close,close,Volume
Date,,,,,,
1993-01-29,43.968750,43.968750,43.750000,43.937500,24.763731,1003200
1993-02-01,43.968750,44.250000,43.968750,44.250000,24.939869,480500
...,...,...,...,...,...,...
2024-06-06,534.979980,535.419983,532.679993,534.659973,534.659973,30808500
2024-06-07,533.659973,536.890015,532.539978,534.010010,534.010010,43195000


## Backtesting.py

### Function to create bands

In [ ]:
def bollinger_band(values, n_windows, k, band_type='upper'):
    
    rolling_std = pd.Series(values).rolling(n_windows).std()
    rolling_mean = pd.Series(values).rolling(n_windows).mean()
    
    if band_type == 'upper':
        band = rolling_mean + rolling_std * k
    elif band_type == 'lower':
        band = rolling_mean - rolling_std * k
    
    return band

### Investment strategy

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

class Bollinger(Strategy):
    N_WINDOWS = 30
    K = 2
    
    def init(self):
        self.upper = self.I(bollinger_band, self.data.Close, self.N_WINDOWS, self.K, band_type='upper')
        self.lower = self.I(bollinger_band, self.data.Close, self.N_WINDOWS, self.K, band_type='lower')
    
    def next(self):
        if crossover(self.lower, self.data.Close) and (self.position.size == 0):
            self.buy()

        elif crossover(self.data.Close, self.upper):
            self.position.close()

### Optimize backtesting

#### Conditions

In [ ]:
df = df_base.loc['2000':'2020']

bt = Backtest(
    df, Bollinger, cash=10_000, commission=0,
    trade_on_close=True, exclusive_orders=True
)

#### Optimization

In [ ]:
params = {
    'N_WINDOWS': range(10, 100, 5),
    'K': [1, 2, 3, 4, 5]
}

results = bt.optimize(**params, maximize='Return [%]', return_heatmap=True)
results

/Users/sota/miniforge3/envs/work/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/15 [00:00<?, ?it/s]

(Start            2000-01-03 00:00:00
 End              2020-12-31 00:00:00
                         ...         
 _equity_curve                    ...
 _trades             Size  EntryBa...
 Length: 30, dtype: object,
 N_WINDOWS  K
 10         1    171.447794
            2    139.342735
                    ...    
 95         4    309.418993
            5    231.159689
 Name: Return [%], Length: 90, dtype: float64)

#### Reports

##### Heatmap

In [ ]:
heatmap = results[1]
df = heatmap.unstack('K')

(df
 .style
    .background_gradient(cmap='Oranges', axis=None)
    .format('{:.2f}')    
)

K,1,2,3,4,5
N_WINDOWS,,,,,
10,171.45,139.34,nan,nan,nan
15,126.38,252.48,156.78,nan,nan
20,68.58,146.29,97.51,nan,nan
25,38.43,169.61,194.87,73.01,nan
30,67.77,110.93,154.28,87.08,nan
35,48.69,69.97,237.00,94.89,nan
40,109.01,54.58,237.00,94.89,nan
45,92.16,33.71,114.06,231.16,nan
50,67.49,26.95,122.19,231.16,nan


##### Table

In [ ]:
results = bt.run()
results.loc[:'Return [%]'].to_frame(name='Value')

,Value
Start,2000-01-03 00:00:00
End,2020-12-31 00:00:00
Duration,7668 days 00:00:00
Exposure Time [%],52.630583
Equity Final [$],21093.488831
Equity Peak [$],21107.488831
Return [%],110.934888


##### Dashboard

In [ ]:
# !python -m pip install bokeh==2.4.3

In [ ]:
import backtesting
backtesting.set_bokeh_output(notebook=True)

In [ ]:
bt.plot(open_browser=False)

Row(id='5037', ...)

##### Individual Trades

In [ ]:
results._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,73,33,51,135.312500,146.343750,805.281250,0.081524,2000-02-18,2000-03-16,27 days
1,76,144,326,142.093750,125.650002,-1249.724884,-0.115725,2000-07-28,2001-04-19,265 days
...,...,...,...,...,...,...,...,...,...,...
34,60,5067,5131,312.649994,303.529999,-547.199707,-0.029170,2020-02-25,2020-05-27,92 days
35,56,5214,5282,322.640015,371.989990,2763.598633,0.152957,2020-09-23,2020-12-30,98 days
